# Imports

In [30]:
import numpy as np
import torch

# Data

$ 
y0 = 3.4  x_{0} - 1.2 x_{2} \\
y1 = 2.7 x_{1} + 4.5
$

In [31]:
n_data = 20
n_f = 3
n_o = 2

x = np.random.uniform(-10, 10, (n_data, n_f))
w0 = np.array([[3.4, 0, -1.2], [0,2.7,0]])
b0 = np.array([0, 4.5])
t = np.dot(x, w0.T) + b0

x = torch.tensor(x)
t = torch.tensor(t)

# Creating a linear regression from scratch

In [60]:
class Linear_Regression:
    def __init__(self, n_features, n_outputs) -> None:
        self.n_f = n_features
        self.n_o = n_outputs
        self.w = torch.rand((self.n_o, self.n_f), dtype=torch.float64, requires_grad=True)
        self.b = torch.rand(self.n_o, dtype=torch.float64, requires_grad=True)

    def predict(self, x):
        return torch.matmul(x, self.w.T) + self.b

    def MSE(self, t, y):
        diff = t - y
        return torch.sum(diff * diff) / diff.numel()

    def train(self, x, t, n_epochs, lr):
        for e in range(n_epochs):
            y = self.predict(x)
            loss = self.MSE(t, y)
            loss.backward()
            with torch.no_grad():
                w -= w.grad * lr
                b -= b.grad * lr
                w.grad.zero_()
                b.grad.zero_()
